# Event Producer 1

Kuah Jia Chen

This program writes records to Kafka topic `AssignmentPartB`. Each record has `key = "climate"` and `value = "climate data"`.

Import necessary libraries

In [1]:
from time import sleep
from json import *
from kafka import KafkaProducer
import random
import datetime as dt
import pandas as pd
from datetime import *

Publish the key value pair using the Kafka producer instance

In [2]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')                              # encode the key to bytes
        value_bytes = bytes(value, encoding='utf-8')                          # encode the value to bytes
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)  # publish the message using the topic_name
        producer_instance.flush()
        # print out successful message once the data is published
        print('Message published successfully. Data: ' + str(data) + " Key:" + key)
    except Exception as ex:
        print('Exception in publishing message.')                  # else, print out error message
        print(str(ex))

Get Kafka producer instance

In [3]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

Find the latest date from the collection created in Part A

In [4]:
from pymongo import MongoClient
from pprint import pprint

# Create a MongoClient/Mongodb connection 
client = MongoClient()

# We will use the database: fit3182_assignment_db
db = client.fit3182_assignment_db

cursor = db.embedded_climate_hotspot.find().sort('date',-1).limit(1)
for document in cursor:
    pprint(document['date'].strftime("%d-%m-%Y"))

'31-12-2021'


In [5]:
if __name__ == '__main__':
   
    topic = 'AssignmentPartB'                                    # the topic name
    
    print('Publishing records..')
    producer = connect_kafka_producer()                          # connect to the kafka producer
    
    climate_streaming = pd.read_csv("climate_streaming.csv")     # read the file for streaming data
    
    current_date = "31-12-2021"                                  # get from above
    
    current_date_obj = datetime.strptime(current_date,'%d-%m-%Y')# change it to date object
    
    constant_station_ID = 123456                                 # provide a constant station ID for each climate data
    
    while True:
        
        current_date_obj += timedelta(days=1)                    # generate new date by adding one day from the previous day
        
        new_date = current_date_obj.strftime('%d-%m-%Y')         # make the new data as string
        
        current_data = climate_streaming.sample(replace=True)    # get random data (with replacement)
        
        current_data['date'] = new_date                          # include the date information in the document
        
        current_data['station'] = constant_station_ID            # include the station id information in the document 
        
        data = dumps(current_data.to_dict('records')[0])         # prepare the data with proper data format by
                                                                 # converting the dictionary to string 
        
        publish_message(producer, topic, 'climate',data)         # insert producer information as key
                                                                 # and publish the key value pair
        
        sleep(10)                                                # publish data every 10 seconds

Publishing records..
Message published successfully. Data: {"latitude": -37.485, "longitude": 148.095, "air_temperature_celcius": 11, "relative_humidity": 45.4, "windspeed_knots": 5.2, "max_wind_speed": 8.9, "precipitation ": " 0.00A", "GHI_w/m2": 98, "date": "01-01-2022", "station": 123456} Key:climate
Message published successfully. Data: {"latitude": -37.091, "longitude": 145.362, "air_temperature_celcius": 12, "relative_humidity": 48.8, "windspeed_knots": 4.2, "max_wind_speed": 7.0, "precipitation ": " 0.00I", "GHI_w/m2": 104, "date": "02-01-2022", "station": 123456} Key:climate
Message published successfully. Data: {"latitude": -34.965, "longitude": 142.031, "air_temperature_celcius": 20, "relative_humidity": 62.6, "windspeed_knots": 10.1, "max_wind_speed": 15.0, "precipitation ": " 0.00I", "GHI_w/m2": 153, "date": "03-01-2022", "station": 123456} Key:climate
Message published successfully. Data: {"latitude": -36.9364, "longitude": 143.4996, "air_temperature_celcius": 14, "relativ

Message published successfully. Data: {"latitude": -37.609, "longitude": 149.32, "air_temperature_celcius": 16, "relative_humidity": 48.3, "windspeed_knots": 9.4, "max_wind_speed": 14.0, "precipitation ": " 0.01G", "GHI_w/m2": 139, "date": "30-01-2022", "station": 123456} Key:climate
Message published successfully. Data: {"latitude": -37.4551, "longitude": 143.7902, "air_temperature_celcius": 9, "relative_humidity": 37.9, "windspeed_knots": 9.1, "max_wind_speed": 15.9, "precipitation ": " 0.00G", "GHI_w/m2": 85, "date": "31-01-2022", "station": 123456} Key:climate
Message published successfully. Data: {"latitude": -36.4422, "longitude": 141.42700000000002, "air_temperature_celcius": 15, "relative_humidity": 48.1, "windspeed_knots": 7.9, "max_wind_speed": 12.0, "precipitation ": " 0.00G", "GHI_w/m2": 131, "date": "01-02-2022", "station": 123456} Key:climate
Message published successfully. Data: {"latitude": -37.614000000000004, "longitude": 143.451, "air_temperature_celcius": 19, "relat

KeyboardInterrupt: 